# Get published matrisomes from the MatrisomeProject
The Matrisome project providex excel tables that describe the matrisomes of a number of species.
I downloaded these tables to the Subfolder MatrisomProject_tables

## Clean table for further processing
Each of the matrisomes provides different inforamtion about the proteins of the matrisome. 

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-1wig">Species</th>
    <th class="tg-1wig">Gene_Symbol</th>
    <th class="tg-1wig">ReSeq_prot_ID</th>
    <th class="tg-1wig">Uniprot_ID</th>
    <th class="tg-1wig">Ensemble_ID</th>
    <th class="tg-1wig">WormBase_ID</th>
    <th class="tg-1wig">Sequence</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">Bos taurus</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Coturnix japonica</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Danio rerio</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Homo sapiens</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Mus musculus</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Drosophila melanogaster</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Caenorhabditis elegans</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
  </tr>
  <tr>
    <td class="tg-0lax">Schmidtea mediterranea</td>
    <td class="tg-baqh">Yes</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">-</td>
    <td class="tg-baqh">Yes</td>
  </tr>
</tbody>
</table>

I copied this information to a new excel file that only contains these columns
However, not all the information is up to date and some of the IDs provided are deprecated

In [48]:
import requests, urllib
from Bio import SeqRecord, Seq, SeqIO, Entrez
from pathlib import Path
from io import StringIO
import pandas as pd
import re
import time
import logging
from tqdm import tqdm

Entrez.email = # Enter your Email here

# Configure logging
log_file = 'matrisome_pipeline.log'
logging.basicConfig(filename=log_file, level=logging.INFO,
                    format='%(asctime)s [%(levelname)s]: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')



In [49]:
#retrival function
def get_WormBase_sequences(WormBase_IDs):
    protein_records = []
    for WormBase_ID in WormBase_IDs:
        time.sleep(0.333)
        logging.info(f"Retrieving sequences for WormBase ID: {WormBase_ID}")
        
        gene_url = f"http://rest.wormbase.org/rest/widget/gene/{WormBase_ID}/sequences"
        response = requests.get(gene_url)
        
        if response.status_code == 200:
            gene_data = response.json()
            gene_models = gene_data['fields']['gene_models']['data']['table']
            
            
            for isoform in gene_models:
                protein_id = isoform['protein']['id']
                protein_label = isoform['protein']['label']
                
                protein_url = f"http://rest.wormbase.org/rest/field/protein/{protein_id}/sequence"
                response = requests.get(protein_url)
                
                if response.status_code == 200:
                    protein_data = response.json()
                    sequence = protein_data['sequence']['data']['sequence']
                    
                    seq_record = SeqRecord.SeqRecord(Seq.Seq(sequence))
                    seq_record.id = protein_id
                    seq_record.name = protein_label
                    seq_record.description = f"Protein ID: {protein_id}, Gene Name: {gene_data['fields']['name']['data']['label']}"
                    
                    protein_records.append(seq_record)
            
            logging.info(f"Retrieved {len(protein_records)} protein records for WormBase ID: {WormBase_ID}")
            return protein_records
        else:
            logging.warning(f"Error retrieving sequences for WormBase ID {WormBase_ID}. Status code: {response.status_code}")
            return []

#---------------------------------------------------------------------------------------------
def get_FlyBase_sequences(gene_ids):
    sequences = []
    for gene_id in gene_ids:
        time.sleep(0.333)
        logging.info(f"Retrieving sequences for FlyBase ID: {gene_id}")
        
        url = f"http://api.flybase.org/api/v1.0/sequence/id/{gene_id}/FBpp"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            result = data['resultset'].get('result', [])
            if result:
                
                for entry in result:
                    seq_record = SeqRecord.SeqRecord(Seq.Seq(entry['sequence']), id=entry['id'], description=entry['description'])
                    sequences.append(seq_record)
                logging.info(f"Retrieved {len(sequences)} sequences for FlyBase ID: {gene_id}")
                return sequences
            else:
                logging.warning(f"No sequences found for the given FlyBase ID: {gene_id}")
                return []
        else:
            logging.warning(f"Error retrieving data for FlyBase ID {gene_id}. Status code: {response.status_code}")
            return []
    
#---------------------------------------------------------------------------------------------
def retrieve_RefSeq_sequences(RefSeq_IDs):
    logging.info(f"Retrieving RefSeq sequences for IDs: {RefSeq_IDs}")
    RefSeq_IDs = [id_.rstrip('.') for id_ in RefSeq_IDs]  # remove trailing "."
    sequences = []

    try:
        handle = Entrez.efetch(db="protein", id=",".join(RefSeq_IDs), rettype="fasta", retmode="text")
        records = SeqIO.parse(handle, "fasta")
        sequences = list(records)
        handle.close()
        logging.info(f"Retrieved {len(sequences)} sequences")
    except urllib.error.HTTPError as e:
        logging.error(f"HTTP Error occurred: {e}")
        pass
    except Exception as e:
        logging.error(f"An error occurred: {e}")

    return sequences

#---------------------------------------------------------------------------------------------
def retrieve_uniprot_sequences(uniprot_IDs):
    time.sleep(0.333)
    logging.info(f"Retrieving sequences for Uniprot: {uniprot_IDs}")
    records = []
    
    for uniprot_id in uniprot_IDs:
        if len(uniprot_id) < 4:
            continue
        current_url = f"http://www.uniprot.org/uniprot/{uniprot_id}.fasta"
        response = requests.get(current_url)
        cData = response.text

        if not cData:
            continue

        try:
            pSeq = SeqIO.read(StringIO(cData), 'fasta')
            records.append(pSeq)
            logging.info(f"Retrieved sequence for UniProt ID: {uniprot_id}")
        except ValueError as e:
            logging.warning(f"Error reading sequence for UniProt ID {uniprot_id}: {e}")

    return records

#---------------------------------------------------------------------------------------------
def retrieve_ensembl_sequence(ensemble_ids):
    records = []
    for ensemble_id in ensemble_ids:
        time.sleep(0.333)
        logging.info(f"Retrieving sequences for Ensembl: {ensemble_id}")
        server = "https://rest.ensembl.org"
        ext = f"/sequence/id/{ensemble_id}?type=protein;multiple_sequences=TRUE"
        r = requests.get(server+ext,headers={"Content-Type": "text/x-fasta"})
        cData = r.text
        if not cData:
            continue

        # Parse the FASTA data
        try:
            sequence = SeqIO.parse(StringIO(cData), 'fasta')
            records.extend(sequence)
            logging.info(f"Retrieved sequence for UniProt ID: {ensemble_id}")
        except ValueError as e:
            logging.warning(f"Error reading sequence for UniProt ID {ensemble_id}: {e}")
    return records


#---------------------------------------------------------------------------------------------
def find_gene_symbol_sequences(gene_symbol, organism):
    time.sleep(0.333)
    logging.info(f"Retrieving sequences for : {gene_symbol} in {organism}")
    try:
        query = f'{gene_symbol} AND {organism} [ORGN]'
        handle = Entrez.esearch(db="protein", term=query)
        record = Entrez.read(handle)
        handle.close()

        if record['IdList']:
            handle = Entrez.efetch(db="protein", id=",".join(record['IdList']), rettype="fasta", retmode="text")
            records = list(SeqIO.parse(handle, "fasta"))
            handle.close()
            logging.info(f"Retrieved sequences for gene symbol: {gene_symbol}, organism: {organism}")
            return records

    except Exception as e:
        logging.warning(f"Error retrieving sequences for gene symbol: {gene_symbol}, organism: {organism}. Error: {e}")
    
    return None

#---------------------------------------------------------------------------------------------
def create_fasta_records(sequence, gene_symbol):
    records = []
    record = SeqRecord.SeqRecord(Seq.Seq(sequence), id=gene_symbol, description='', name=gene_symbol)
    record.annotations["molecule_type"] = "protein"
    records.append(record)
    
    # Log the creation of fasta record
    logging.info(f"Created fasta record for gene symbol: {gene_symbol}")
    
    return records

Iterate through all species and genes and look up all possible seq records

## Read the tables of genes  

In [50]:
#define where the input files are
input_folder = Path(r"E:\mesoglea_protein_pipeline\published_input")
matrisome_folder = input_folder / 'modified_tables'

In [51]:
def get_xlsx_files_and_read(folder_path):
    data_dict = {}

    for file_path in Path(folder_path).rglob('*.xlsx'):
        species = file_path.stem
        df = pd.read_excel(file_path)
        df = df.set_index("gene_symbol")
        df["deprecated"] = False
        data_dict[species] = df
        
        # Log the loading of a DataFrame
        logging.info(f"Loaded DataFrame for species: {species}")

    return data_dict

matrisome_dict = get_xlsx_files_and_read(matrisome_folder)

In [52]:
def get_gene_sequences(row, species, index, gene_seqs_folder):
    sequences = []

    if Path.is_file(gene_seqs_folder / f"{index}.fasta"):
        sequences = list(SeqIO.parse(gene_seqs_folder / f"{index}.fasta","fasta"))
        if len(sequences)>0:
            return sequences

    if 'sequence' in row and row["sequence"] and isinstance(row['sequence'], str):
        sequences.extend(create_fasta_records(row['sequence'], index))
        logging.info(f"Created fasta records for gene symbol: {index}")

    if 'uniprot_ID' in row and isinstance(row['uniprot_ID'], str) and len(row['uniprot_ID'])>3 :
        uniprot_ids = re.split(r'[:;]', row['uniprot_ID'])
        sequences.extend(retrieve_uniprot_sequences(uniprot_ids))
        logging.info(f"Retrieved UniProt sequences for gene symbol: {index}")

    if 'RefSeq_ID' in row and len(row['RefSeq_ID'])>3 and isinstance(row['RefSeq_ID'], str):
        refseq_ids = re.split(r'[:;]', row['RefSeq_ID'])
        sequences.extend(retrieve_RefSeq_sequences(refseq_ids))
        logging.info(f"Retrieved RefSeq sequences for gene symbol: {index}")

    if 'WormBase_ID' in row and row['WormBase_ID']and isinstance(row['WormBase_ID'], str):
        refseq_ids = re.split(r'[:;]', row['WormBase_ID'])
        sequences.extend(get_WormBase_sequences(refseq_ids))
        logging.info(f"Retrieved WormBase sequences for gene symbol: {index}")

    if 'FlyBaseID' in row and row['FlyBaseID'] and isinstance(row['FlyBaseID'], str):
        refseq_ids = re.split(r'[:;]', row['FlyBaseID'])
        sequences.extend(get_FlyBase_sequences(refseq_ids))
        logging.info(f"Retrieved FlyBase sequences for gene symbol: {index}")

    if 'ensembl_ID' in row and row['ensembl_ID']and isinstance(row['ensembl_ID'], str):
        refseq_ids = re.split(r'[:;]', row['ensembl_ID'])
        sequences.extend(retrieve_ensembl_sequence(refseq_ids))
        logging.info(f"Retrieved Ensembl sequences for gene symbol: {index}")

    if len(sequences) == 0 and index:
        sequences= find_gene_symbol_sequences(index, species)
        logging.info(f"No sequences found for gene symbol: {index}")

    if sequences:
        logging.info(f"Found {len(sequences)} sequences.")
    else:
        logging.info(f"Found 0 sequences.")
    return sequences

In [56]:
with open('logfile.log', 'w'):
    pass
# Add log messages at relevant points in your code
logging.info("Starting matrisome processing...")

# Iterate through each species and its corresponding DataFrame
for species, main_df in matrisome_dict.items():
    main_df = main_df.dropna(axis=1)
    update= False
    print(f"Working on: {species}")
    print(f"\t expected number of sequences: {main_df.shape[0]}")
    logging.info(f"Working on: {species}")
    logging.info(f"\t expected number of sequences: {main_df.shape[0]}")

    # Check if a _missing.csv file exists for the species
    missing_path = matrisome_folder / "missing" / f"{species}_missing.csv"
    if missing_path.is_file():
        # If the file exists, read it as a DataFrame
        df = pd.read_csv(missing_path)
        df = df.set_index("gene_symbol")
        print(f"Found missing data for {species}. Continuing with {df.shape[0]} rows.")
        logging.info(f"Found missing data for {species}. Continuing with {df.shape[0]} rows.")
        update = True
    else:
        df = main_df
        print("Run normally.")

    df['deprecated'] = df['deprecated'].astype('object')    
    best_guess_matrisome = []  # Initialize the list to hold the longest sequences
    missing = pd.DataFrame(columns=df.columns)  # Initialize the DataFrame for missing rows
    missing_ids = list()
    gene_seqs_folder = matrisome_folder / "gene_seqs" / species
    gene_seqs_folder.mkdir(parents=True, exist_ok=True)    

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        tax = species.split("_")
        species_name = " ".join([tax[-2], tax[-1]])
        sequences = get_gene_sequences(row, species_name,index,gene_seqs_folder)

        if sequences:
            # Find the longest sequence
            longest_sequence = max(sequences, key=lambda x: len(x.seq))
            best_guess_matrisome.append(longest_sequence)

            # Create a subfolder "gene_seqs"


            # Create a fasta file for the gene
            clean_id = str(index).replace("/","_").replace("\\","_")
            gene_fasta_path = gene_seqs_folder / f"{clean_id}.fasta"
            main_df.loc[index,"potential SeqIDs"] = ";".join([seq.id for seq in sequences])
            main_df.loc[index,"longest Sequence"] = str(longest_sequence.seq)
            SeqIO.write(sequences, gene_fasta_path, "fasta")

            # Remove the deprecated tag from the main df
            main_df.loc[index,"deprecated"] = ""

        else:
            # If no sequences found, add the row to the missing DataFrame
            missing.loc[index] = row
            missing_ids.append(index)

    # Create a fasta file for the best guess matrisome
    best_guess_path = matrisome_folder/"retrived_sequences"
    best_guess_path.mkdir(parents=True, exist_ok=True)
    if best_guess_matrisome:
        best_guess_fasta_path = best_guess_path / f"{species}.fasta"
        if update and Path.is_file(best_guess_fasta_path):
            print(f"Adding {len(best_guess_matrisome)}")
            exsisting_seqs = list(SeqIO.parse(best_guess_fasta_path, "fasta"))
            best_guess_matrisome = best_guess_matrisome + exsisting_seqs
            assert len(best_guess_matrisome) > len(exsisting_seqs)
            missing_path.unlink()# delete the old tabel of missing sequences
        SeqIO.write(best_guess_matrisome, best_guess_fasta_path, "fasta")
        logging.info(f"\tFound: {len(best_guess_matrisome)} Sequences")
        print(f"\tFound: {len(best_guess_matrisome)} Sequences")
    
    # Save the "missing" DataFrame as a CSV file
    missing_path = matrisome_folder/"missing"
    missing_path.mkdir(parents=True, exist_ok=True)
    if missing.shape[0]:
        missing_csv_path = missing_path/ f"{species}_missing.csv"
        missing['gene_symbol'] = missing_ids
        missing.to_csv(missing_csv_path, index=False)
logging.shutdown()

Working on: Chordata_Vertebrata_Bos_taurus
	 expected number of sequences: 1087
Found missing data for Chordata_Vertebrata_Bos_taurus. Continuing with 1 rows.


100%|██████████| 1/1 [00:00<00:00, 1000.55it/s]


Working on: Chordata_Vertebrata_Coturnix_japonica
	 expected number of sequences: 757
Found missing data for Chordata_Vertebrata_Coturnix_japonica. Continuing with 69 rows.


100%|██████████| 69/69 [00:56<00:00,  1.23it/s]


Working on: Chordata_Vertebrata_Danio_rerio
	 expected number of sequences: 1015
Found missing data for Chordata_Vertebrata_Danio_rerio. Continuing with 42 rows.


...

	Found: 641 Sequences
Working on: Nematoda_Chromadorea_Caenorhabditis_elegans
	 expected number of sequences: 719
Found missing data for Nematoda_Chromadorea_Caenorhabditis_elegans. Continuing with 2 rows.


100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


Working on: Platyhelminthes_Tricladida_Schmidtea_mediterranea
	 expected number of sequences: 256
Run normally.


100%|██████████| 256/256 [00:02<00:00, 111.72it/s]


	Found: 256 Sequences


In [57]:
for species,df in matrisome_dict.items():
    df.to_csv(f"filled_{species}.csv")

In [38]:
matrisome_dict["Chordata_Vertebrata_Bos_taurus"]

,deprecated,potential SeqIDs,longest Sequence
gene_symbol,,,
COL10A1,,sp|P23206.1|COAA1_BOVIN;NP_777059.1;DAA26300.1...,MLPQTALLLLMSLNLVHGVFYTERYQTPTGIKGPPSNTKTQFFIPY...
COL11A1,,XP_059737868.1;XP_059737865.1;sp|Q28083.1|COBA...,MERWSSRWKTKRWLWDFTITTLALTFLFQAREVRGADPVDVLKALD...
COL11A2,,XP_059736316.1;sp|Q32S24.1|COBA2_BOVIN;NP_0010...,MERCSRCHHLLLLVLLLLWLSAAPAWAGTAPVDVLRALRFPALPDG...
COL12A1,,XP_059745578.1;XP_059745577.1;XP_059745576.1;s...,MRIRLPPALAALGAALLLSSIEAEVEPPSDLNFKIIDENTVHMSWA...
COL13A1,,XP_059738581.1;XP_059738580.1;XP_059738579.1;X...,MVAERSRKATAAGARGPAELDAPGTVALAVAPAERCARLPSPGSCG...
...,...,...,...
61,,pdb|8GRR|L;pdb|6NO7|D;pdb|6NHG|G;pdb|6NKN|N;pd...,MSSQVEHPAGGYKKLFETVEELSSPLTAHVTGRIPLWLTGSLLRCG...
62,,pdb|6NHG|G;pdb|6NKN|N;pdb|6NMP|N;pdb|6NMF|N;pd...,MNPSASFLAGRQNIGSEVEISTIEKQRKELQLLIGELKDRDRELND...
63,,pdb|6MRQ|A;pdb|6NHG|H;pdb|6NKN|N;pdb|6NMP|N;pd...,MSSGLWSQEKVTSPYWEERIFYLLLQECSVTDKQTQKLLKVPKGSI...
